In [1]:
import pandas as pd 
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
#path = f"x264/sampledConfigurations_{name_of_technique}_t3.csv"
#lst = ['distBased', 'divDistBased', 'henard', 'random', 'solverBased', 'twise']

path = r"sampledConfigurations_divDistBased_t3.csv"


df = pd.read_csv(path, sep=';')


In [2]:
df

,root,time_passes,gvn,instcombine,inline,jump_threading,simplifycfg,sccp,print_used_types,ipsccp,iv_users,licm,Performance
0,1,1,0,0,0,0,0,0,0,0,1,0,210.340000
1,1,1,1,0,0,0,0,0,0,0,1,0,219.153333
2,1,1,0,0,0,1,0,0,0,0,1,0,199.683333
3,1,1,0,0,0,0,1,0,0,0,1,0,200.380000
4,1,1,0,0,0,0,0,0,1,0,1,0,203.180000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,1,1,1,1,1,1,0,1,0,1,0,1,262.220000
171,1,1,1,1,1,1,0,1,0,1,1,1,268.410000
172,1,1,1,1,1,1,0,1,1,0,1,1,261.363333
173,1,1,1,1,1,1,0,1,1,1,1,1,267.316667


In [9]:
df_time = pd.DataFrame(columns=['SampleAlgorithm','%FeatureSelection','#Features','FeatureImportance','%Configurations','Dummy','StartTime', 'EndTime'])
exec_time = ['DistBased',0.3,18,True,0.1,True,pd.Timestamp.now(),pd.Timestamp.now()]
df_time.loc[df_time.size] = exec_time
exec_time = ['DistBased',0.6,18,False,0.9,True,pd.Timestamp.now(),pd.Timestamp.now()]
df_time.loc[df_time.size] = exec_time

df_time

,SampleAlgorithm,%FeatureSelection,#Features,FeatureImportance,%Configurations,Dummy,StartTime,EndTime
0,DistBased,0.3,18,True,0.1,True,2023-06-09 15:03:59.612579,2023-06-09 15:03:59.612621
8,DistBased,0.6,18,False,0.9,True,2023-06-09 15:03:59.628869,2023-06-09 15:03:59.628900


### Geração de feature importance sem a utilização de hiperparâmetros

In [3]:
def run_featureSelection(df, hyperparameters=None, path=None):
    X = df.drop('Performance', axis=1)
    y = df['Performance']

    if hyperparameters is None:
        rf = RandomForestRegressor()
    else:
        rf = RandomForestRegressor(**hyperparameters)
    
    rf.fit(X, y)

    importances = rf.feature_importances_

    feature_importances = pd.DataFrame(importances, index = X.columns, columns=['importance']).sort_values('importance', ascending=False)

    print(feature_importances)
    
    if hyperparameters is None:
        if path is None:
            feature_importances.to_csv("feature_importance.csv", header=True)
        else:
            feature_importances.to_csv("feature_importance_"+path+".csv", header=True)
    else:
        if path is None:
            feature_importances.to_csv("feature_importanceHP.csv", header=True)
        else:
            feature_importances.to_csv("feature_importanceHP_"+path+".csv", header=True)


run_featureSelection(df)

                  importance
gvn                 0.311751
inline              0.265344
licm                0.264886
instcombine         0.087254
jump_threading      0.014304
print_used_types    0.013949
iv_users            0.012312
ipsccp              0.011334
sccp                0.010700
simplifycfg         0.008165
root                0.000000
time_passes         0.000000


### Geração de feature importance com a utilização de hiperparâmetros

In [4]:
hyperparams_RF = {
    "max_depth": 25,
    "n_estimators": 10,
    "n_jobs" : -1
}

run_featureSelection(df, hyperparams_RF)

                  importance
gvn                 0.333793
inline              0.283232
licm                0.226634
instcombine         0.083943
iv_users            0.015208
jump_threading      0.014447
print_used_types    0.013598
ipsccp              0.012097
sccp                0.010007
simplifycfg         0.007039
root                0.000000
time_passes         0.000000


In [5]:
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

for config_name in samples_config:
    for t in [1,2,3]:
        path = r"sampledConfigurations_"+config_name+"_t"+str(t)+".csv"
        path_name = config_name+"_t"+str(t)
        df = pd.read_csv(path, sep=';')
        
        run_featureSelection(df, None, path_name)
        run_featureSelection(df, hyperparams_RF, path_name)

                  importance
gvn                 0.516688
instcombine         0.111324
inline              0.109439
jump_threading      0.079660
sccp                0.065863
licm                0.045539
iv_users            0.031822
ipsccp              0.017044
print_used_types    0.012737
simplifycfg         0.009885
root                0.000000
time_passes         0.000000
                  importance
gvn                 0.583714
instcombine         0.118893
jump_threading      0.078673
iv_users            0.076993
inline              0.050680
licm                0.033128
sccp                0.027991
simplifycfg         0.025355
print_used_types    0.003751
ipsccp              0.000822
root                0.000000
time_passes         0.000000
                  importance
licm                0.300096
gvn                 0.267411
inline              0.205252
instcombine         0.130750
jump_threading      0.023784
print_used_types    0.022619
sccp                0.020157
simplifycfg   

                  importance
gvn                 0.308390
licm                0.260668
inline              0.198531
instcombine         0.167685
simplifycfg         0.016520
sccp                0.012805
jump_threading      0.011059
iv_users            0.009717
ipsccp              0.008844
print_used_types    0.005782
root                0.000000
time_passes         0.000000
                  importance
gvn                 0.298892
licm                0.254512
inline              0.208798
instcombine         0.163927
simplifycfg         0.018846
iv_users            0.013145
sccp                0.013086
ipsccp              0.011475
jump_threading      0.010183
print_used_types    0.007136
root                0.000000
time_passes         0.000000
                  importance
instcombine         0.509011
ipsccp              0.304969
inline              0.107756
simplifycfg         0.040972
print_used_types    0.037292
root                0.000000
time_passes         0.000000
gvn           